## Вводная

Вы получили основные навыки обработки данных, теперь пора испытать их на практике. Сейчас вам предстоит заняться задачей классификации.

Представлен известный датасет “Титаник”, и вашей задачей будет обучить модель таким образом, чтобы  по определенным признакам была возможность максимально уверенно предсказать - выживет или умрёт пассажир (столбец “Survived”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:
1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на имена пассажиров. Подумайте, что можно извлечь полезного оттуда
6. Использование профайлера вам поможет.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

Желаю успеха :)

> Спасибо ;-)

## Начало работы

Начнём с того, что подготовим себе окружение и загрузим данные

In [1]:
%matplotlib inline

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pandas_profiling

Нам даны три файла:
- `test.csv` - тестовый датасет
- `train.csv` - тренировочные датасет
- `gender_submission.csv` - датасет выживших пассажиров по колонке PassengerId

In [2]:
df_test = pd.read_csv('./data/test.csv')
df_train = pd.read_csv('./data/train.csv')
df_gs = pd.read_csv('./data/gender_submission.csv')

df_test.shape, df_train.shape, df_gs.shape

((418, 11), (891, 12), (418, 2))

С ходу можно сказать, что в тестовом датасете не хватает столбца и он в два раза меньше тренировочного. Так же, датасет выживших имеет такое же количество записей, что и тестовый датасет. Посмотрим на датасеты и постараемся разобраться, в чём дело 

In [3]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_gs.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Определенно, разница в данных между тестовым и тренировочным датасетом заключается в том, что признак выжившего пассажира удалили из тестового датасета и сохранили его в отдельный файл

Данные у нас есть. Целевое значение `Survived`.

Теперь необходимо внести немного ясности и понять, как какая переменная могла повлиять на целевое значение.

Для этого я приведу описание каждого из параметров:
- "PassengerId" - идентификатор пассажира в пределах датасета
- "Pclass" - класс билета (1 - 1й, 2 - 2й, 3 - 3й)
- "Name" - имя и принятое обращение к пассажиру
- "Sex" - пол
- "Age" - возраст в годах
- "SibSp" - количество братьев/сестёр или супругов на борту
- "Parch" - количество детей или родителей на борту
- "Ticket" - номер билета
- "Fare" - тариф
- "Cabin" - номер каюты
- "Embarked" - порт, в котором пассажир зашёл на борт (C = Шербур, Q = Квинстаун, S = Саутгемптон)
- "Survived" - признак, выжил ли пассажир (0 - нет, 1 - да)

"SibSp" - учёт родных братьев/сестёр родных и сводных, супругов. Любовники не были учтены
"Parch" - учёт родных родителей и детей (родных/приёмных). Некоторые дети путешествовали только с нянечкой, поэтому признак может быть равен нулю у них

**Описание было взято на kaggle.com, после чего сайт был закрыт сразу ради сохранения спортивного интереса**

## Анализ переменных

Я проведу анализ переменных и приму решение о каждой из них, как они могут повлиять итоговый результат, нужна ли нам та или иная переменная и можно ли сгенерировать новые переменные.

### PassengerId
Идентификатор пассажира в рамках датасета никаким образом не влияет на результат

При обучении, этот параметр я удалю

### Pclass
Уверенно могу сказать, что класс билета влияет на результат, ведь такой корабль просто не мог делать поправку на статус пассажира при спасении.

Данный параметр категориальный и к нему можно применить One Hot Encoding

### Name
В данном признаке помимо имён содержится обращение к пассажиру, вроде "Mr.", "Mrs." и т.д., думаю это можно использовать как дополнительный признак при помощи One Hot Encoding

### Sex
Определенно важный признак, т.к. даже есть выражение такое: "Первыми спасают женщин и детей, чтобы мужчины могли спокойно подумать". Концовка фразы, конечно, в шутку, но данный признак, я уверен, играет важную роль

### SibSp
Как мне кажется, этот признак мог определять поведение человека во время крушения. Не каждый человек готов бросить своих родных и спасаться самому, поэтому он мог возвращаться за ними в спешке и суматохе, что повлияло на его шансы на спасение.

### Parch
Данный признак тоже важен, т.к. это могло спровоцировать самопожертвование. Я уверен, что многие взрослые могли отдать жизнь, лишь бы спасли их чадо

### Ticket
Я не вижу в номере билета никаких закономерностей. Класс пассажира определяется отдельной переменной, каюта тоже, где он сел на корабль тоже

Этот признак я удалю

### Fare
Стоимость тарифа пассажира может рассказать нам о том, что пассажир был богатым. Я уверен, что многие пассажиры при спасении пытались подкупить стюардов, сажающих на спасательные шлюпки людей, поэтому признак оставлю как важный

### Cabin
Я не вижу в этом признаке ничего важного. Значения описывающие номер кабины мне ни о чём не говорят и мне трудно придумать логику, на которую можно завязсяться при работе с этим признаком

Этот признак я удалю

### Embarked
Признак который говорит, где произвёл посадку тот или иной пассажир. Всего в датасете 3 города:
- Шербур - Франция
- Квинстаун - Новая Зеландия
- Саутгемптон - Великобритания

Это может рассказать нам о темпераменте и характере пассажира. На сколько я вижу всё это, то скорее всего люди из Англии и Франции медленнее спасались в виду "перевоспитанности" одних и "мягкотелости" других. А вот люди из Новой Зеландии спасались значительно увереннее, т.к. это потомки заключённых, вывезенных в Новый свет Англичанами во времена колонизации. Но это только если пассажиры одного класса. Если пассажиры были разных классов, то данный признак может наоборот увести пассажиров третьей категории в список пассажиров с меньшими регалиями, что негативно сказалось бы на их шансе выжить.

Данный признак я оставляю и применяю к нему One Hot Encoding

### Survived
Признак, выжил ли пассажир. Наше целевое значение в булевом представлении

## Предобработка данных

Для нас заранее разбили данные на тестовые и тренировочные, вдобавок от тестовых данных оторвали переменную `Survived`. Я считаю это крайне не удобным, т.к. каждую операцию по дальнейшей подготовке данных выполнить два раза, вдобавок потом как-то придётся мержить оторванную колонку.

Поэтому сперва я хочу вернуть оторванную колонку, соединить датасеты тренировочных и тестовых данных, а уже только потом создавать новые признаки и производить чистку выбросов и т.п.

In [6]:
df_gs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
PassengerId    418 non-null int64
Survived       418 non-null int64
dtypes: int64(2)
memory usage: 6.7 KB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Краткая информация о датасетах показывает, что на этапе слияния `df_gs` и `df_test` проблем возникнуть не должно

In [8]:
df_test = df_test.merge(df_gs, on='PassengerId')
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [9]:
df_common = pd.concat([df_test, df_train], sort=False)
df_common

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


Выглядит не плохо. Теперь избавимся от ненужных нам столбцов "PassengerId", "Ticket", "Cabin"

In [10]:
df_common = df_common.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1)
df_common

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q,0
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S,1
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q,0
3,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S,0
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S,1
...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S,0
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S,1
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,S,0
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C,1


Лишние столбцы отсутствуют, теперь можно пойти по намеченой стратегии и начать извлекать данные из столбцов.

Для этого мне придётся научиться использовать `OeHotEncoder`

Примерно 100 попыток спустя...

Так как каждый столбец, который мы преобразовываем, будет подвергаться примерно одним и тем же действиям, то я решил написать функцию, выполняющую эти действия

In [11]:
def one_hot_encode_new_columns(df: pd.DataFrame, col_name: str):
    enc = OneHotEncoder(categories='auto')
    
    encoded_data = enc.fit_transform(
        np.array( df[col_name] ).reshape(-1, 1)
    ).todense()
    
    encoded_feature_names = list(map(lambda val: re.sub(r'^.+_', f'{col_name}_', val), enc.get_feature_names()))
    
    return pd.DataFrame(data=encoded_data, columns=encoded_feature_names)

In [12]:
pd.concat([
    df_common,
    one_hot_encode_new_columns(df_common, 'Pclass'),
    one_hot_encode_new_columns(df_common, 'Sex'),
    one_hot_encode_new_columns(df_common, 'Embarked'),
])

ValueError: Input contains NaN

Я специально не стал удалять стек-трейс, чтобы плавно перейти к тому, что сперва нужно было обработать все битые значения, а потом производить слияние. Так же, я должен обработать переменную Name, из которой я хочу извлечь новый признак.

Пожалуй, сперва обработаем поле Name, т.к. его в последствие придётся преобразовывать при помощи OneHotEncoder

In [13]:
def cut_appeal(name: str) -> str:
    return list(filter(re.compile('.+\.').match, name.split()))[0]

df_common['Appeal'] = df_common['Name'].apply(cut_appeal)
df_common.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,Appeal
0,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q,0,Mr.
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S,1,Mrs.
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q,0,Mr.
3,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S,0,Mr.
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S,1,Mrs.


Получилось не плохо! Единственное что меня теперь беспокоит, что будет недостаток некоторых новых категорий. Проверю количество уникальных значений новой переменной

In [14]:
appeals = df_common.groupby(by='Appeal').agg(count=('Appeal', 'count')).reset_index()
appeals

,Appeal,count
0,Capt.,1
1,Col.,4
2,Countess.,1
3,Don.,1
4,Dona.,1
5,Dr.,8
6,Jonkheer.,1
7,Lady.,1
8,Major.,2
9,Master.,61


Действительно, моя догадка была верна. Я думаю, что, к сожалению, придётся избавиться от всех записей, где переменная 'Appeal' встречается меньше 10 раз

In [15]:
filtered_appeals = list(appeals[appeals['count'] > 10]['Appeal'])
filtered_appeals

['Master.', 'Miss.', 'Mr.', 'Mrs.']

In [16]:
df_common = df_common[df_common['Appeal'].isin(filtered_appeals)]
df_common.shape

(1275, 10)

Теперь переменная `Name` нам не нужна

In [17]:
df_common = df_common.drop(['Name'], axis=1)
df_common.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,Appeal
0,3,male,34.5,0,0,7.8292,Q,0,Mr.
1,3,female,47.0,1,0,7.0000,S,1,Mrs.
2,2,male,62.0,0,0,9.6875,Q,0,Mr.
3,3,male,27.0,0,0,8.6625,S,0,Mr.
4,3,female,22.0,1,1,12.2875,S,1,Mrs.


Отлично! Теперь у нас остались только рабочие значения и мы можем проверить датасет на наличие битых значений

In [18]:
df_common.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1275 entries, 0 to 890
Data columns (total 9 columns):
Pclass      1275 non-null int64
Sex         1275 non-null object
Age         1014 non-null float64
SibSp       1275 non-null int64
Parch       1275 non-null int64
Fare        1274 non-null float64
Embarked    1273 non-null object
Survived    1275 non-null int64
Appeal      1275 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 99.6+ KB


Итого - колонки "Age", "Fare", "Embarked" имеют битые значения.

Но есть пара проблем:
1. Переменная "Age" имеет очень много битых значений
2. Переменная "Embarked" категориальная и заполнять её каким-нибудь значением по моде может быть не самым лучшим решением. Впрочем, удалять их тоже не лучшее решение, поэтому я заполню их по моде

In [19]:
df_common.loc[df_common[df_common['Embarked'].isna()].index]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,Appeal
61,2,male,32.0,0,0,13.5,S,0,Mr.
61,1,female,38.0,0,0,80.0,NaN,1,Miss.
829,1,female,62.0,0,0,80.0,NaN,1,Mrs.


Мистика какая-то... Запись Шредингера какая-то... 61я запись одновременно male и female. Одновременно Mr. и Mrs... Это надо исправлять!

In [20]:
df_common = df_common.reset_index(drop=True)
df_common.shape

(1275, 9)

Теперь заменим битые "Embarked" на моду этой переменной

In [21]:
df_common.loc[df_common[df_common['Embarked'].isna()].index, 'Embarked'] = df_common['Embarked'].mode()[0]

In [22]:
df_common.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 9 columns):
Pclass      1275 non-null int64
Sex         1275 non-null object
Age         1014 non-null float64
SibSp       1275 non-null int64
Parch       1275 non-null int64
Fare        1274 non-null float64
Embarked    1275 non-null object
Survived    1275 non-null int64
Appeal      1275 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 89.8+ KB


Переменную "Fare" я заменю на медиану

In [23]:
df_common.loc[df_common[df_common['Fare'].isna()].index, 'Fare'] = df_common['Fare'].median()

In [24]:
df_common.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 9 columns):
Pclass      1275 non-null int64
Sex         1275 non-null object
Age         1014 non-null float64
SibSp       1275 non-null int64
Parch       1275 non-null int64
Fare        1275 non-null float64
Embarked    1275 non-null object
Survived    1275 non-null int64
Appeal      1275 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 89.8+ KB


Осталось разобраться только с значением переменной "Age", тут всё будет несколько сложнее, так как битых значений много и они могут сильно повлиять на окончательный результат...

Попробуем изучить записи с битым полем "Age"

In [25]:
df_common[df_common['Age'].isna()].describe()

,Pclass,Age,SibSp,Parch,Fare,Survived
count,261.000000,0.0,261.000000,261.000000,261.000000,261.000000
mean,2.647510,NaN,0.486590,0.245211,19.793805,0.291188
std,0.722099,NaN,1.453382,0.953347,27.619055,0.455183
min,1.000000,NaN,0.000000,0.000000,0.000000,0.000000
25%,3.000000,NaN,0.000000,0.000000,7.750000,0.000000
50%,3.000000,NaN,0.000000,0.000000,8.050000,0.000000
75%,3.000000,NaN,0.000000,0.000000,22.358300,1.000000
max,3.000000,NaN,8.000000,9.000000,227.525000,1.000000


Как видно из таблицы выше, основная масса пассажиров, у которых нет упоминания о возрасте - пассажиры 3го класса. Из этих пассажиров выжило только ~30% и таких записей 261...

Определенно, это может очень сильно сказаться на окончательном результате. Поэтому здесь для каждой записи я применю следующий алгоритм:
- определю, к какому "Pclass" относится данная запись
- определю значение поля "Appeal" записи
- отфильтроваф все записи по "Pclass" и "Appeal" в соответствие показателями данной записи, возьму её мединное значение по полю "Age"
- подставлю выбранное значение вместо нашего битого

In [26]:
def calc_age_by_pclass_and_appeal(record, self_df):
    Pclass = record['Pclass']
    Appeal = record['Appeal']
    filtered_records = self_df[(self_df['Pclass'] == Pclass) & (self_df['Appeal'] == Appeal)]
    record['Age'] = filtered_records['Age'].median()
    
    return record
    
df_common.loc[df_common[df_common['Age'].isna()].index] = df_common[df_common['Age'].isna()].apply(calc_age_by_pclass_and_appeal, axis=1, self_df=df_common)

In [27]:
df_common.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 9 columns):
Pclass      1275 non-null int64
Sex         1275 non-null object
Age         1275 non-null float64
SibSp       1275 non-null int64
Parch       1275 non-null int64
Fare        1275 non-null float64
Embarked    1275 non-null object
Survived    1275 non-null int64
Appeal      1275 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 89.8+ KB


Отлично! С битыми значениями покончено. Теперь можно повторить наш эксперемент с OneHotEncoding

In [28]:
cats_col_names = ['Pclass', 'Sex', 'Embarked', 'Appeal']

df_cats = pd.concat([
    df_common,
    *map(lambda col_name: one_hot_encode_new_columns(df_common, col_name), cats_col_names)
], sort=False, axis=1)

df_cats.drop(cats_col_names, axis=1, inplace=True)

df_cats.head()

,Age,SibSp,Parch,Fare,Survived,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Appeal_Master.,Appeal_Miss.,Appeal_Mr.,Appeal_Mrs.
0,34.5,0,0,7.8292,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,47.0,1,0,7.0000,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,62.0,0,0,9.6875,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,27.0,0,0,8.6625,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,22.0,1,1,12.2875,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


Теперь самое время проверить данные на выбросы. Для этого я воспользуюсь пандас профайлером

In [29]:
df_cats.profile_report(style={ 'full_width': True })

Я не стал сокращать кол-во переменных в исследовании, чтобы посмотреть наиболее полную картину вокруг них.

## Нулевые значения

В некоторых не булевых переменных есть нулевые значения, но это критично только для переменной "Fare". Их придётся заменить на медиану.

Остальные переменные не требуют обработки нулевых значений

## Выбросы
Я посмотрел отчёт по всем переменным и не нашёл ни одного выброса. Есть "зашкаливающие" значения в переменной "Fare", но их всего 2, оба они - билеты первого класса. Я думаю, что их можно оставить и это не сильно повлияет на конечный результат

## Итого
После исследования я понимаю, что мне требуется лишь избавиться от всех нулевых "Fare" и заменить их на медиану, учитывая его "Pclass"

In [30]:
df_cats.loc[df_cats[(df_cats['Fare'] == 0) & (df_cats['Pclass_1'] == 1)].index] = df_cats[df_cats['Pclass_1'] == 1]['Fare'].median()
df_cats.loc[df_cats[(df_cats['Fare'] == 0) & (df_cats['Pclass_2'] == 1)].index] = df_cats[df_cats['Pclass_2'] == 1]['Fare'].median()
df_cats.loc[df_cats[(df_cats['Fare'] == 0) & (df_cats['Pclass_3'] == 1)].index] = df_cats[df_cats['Pclass_3'] == 1]['Fare'].median()

In [31]:
df_cats['Fare'].describe()

count    1275.000000
mean       33.337184
std        51.818566
min         3.170800
25%         7.910400
50%        14.458300
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

Выглядит не плохо, теперь можно заняться основной "магией" и приняться за обучение модели.

In [32]:
X = df_cats.drop(['Survived'], axis=1)
Y = df_cats['Survived']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [33]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((892, 16), (383, 16), (892,), (383,))

In [34]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

ValueError: Unknown label type: 'continuous'

В "интернетах" пишут, что эта ошибка возникает в связи с тем, что я пытаюсь сделать классификацию по значениям с плавющей точкой. Приведём всё к `int` грубым образом

In [35]:
for col_name in df_cats.columns:
    df_cats[col_name] = df_cats[col_name].astype('int')
    
df_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 17 columns):
Age               1275 non-null int32
SibSp             1275 non-null int32
Parch             1275 non-null int32
Fare              1275 non-null int32
Survived          1275 non-null int32
Pclass_1          1275 non-null int32
Pclass_2          1275 non-null int32
Pclass_3          1275 non-null int32
Sex_female        1275 non-null int32
Sex_male          1275 non-null int32
Embarked_C        1275 non-null int32
Embarked_Q        1275 non-null int32
Embarked_S        1275 non-null int32
Appeal_Master.    1275 non-null int32
Appeal_Miss.      1275 non-null int32
Appeal_Mr.        1275 non-null int32
Appeal_Mrs.       1275 non-null int32
dtypes: int32(17)
memory usage: 84.8 KB


In [45]:
X = df_cats.drop(['Survived'], axis=1)
Y = df_cats['Survived']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [46]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
clf.score(X_test, Y_test)

0.856396866840731

**Итого - точность предсказания модели получилась примерно равна 85.6%**

Наш профайлер любезно подсказывал нам, что он бы исключил значение "Sex_male", т.к. оно сильно коррелирует с значением "Appeal_Mr."

Попробуем исключить данный признак и посмотреть, как это повлияет на результаты предсказаний

In [48]:
df_cats = df_cats.drop(['Sex_male'], axis=1)
df_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 16 columns):
Age               1275 non-null int32
SibSp             1275 non-null int32
Parch             1275 non-null int32
Fare              1275 non-null int32
Survived          1275 non-null int32
Pclass_1          1275 non-null int32
Pclass_2          1275 non-null int32
Pclass_3          1275 non-null int32
Sex_female        1275 non-null int32
Embarked_C        1275 non-null int32
Embarked_Q        1275 non-null int32
Embarked_S        1275 non-null int32
Appeal_Master.    1275 non-null int32
Appeal_Miss.      1275 non-null int32
Appeal_Mr.        1275 non-null int32
Appeal_Mrs.       1275 non-null int32
dtypes: int32(16)
memory usage: 79.8 KB


In [58]:
X = df_cats.drop(['Survived'], axis=1)
Y = df_cats['Survived']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [59]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [60]:
clf.score(X_test, Y_test)

0.8746736292428199

**Итого - точность предсказания модели возвросла примерно на 1.8% до 87.4%**